In [2]:
from transformers import AutoTokenizer, BertModel
import torch
import pandas as pd

C:\Users\JellevanAs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
#Setting up the BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [5]:

df_model = pd.read_csv(r"C:\\Users\\JellevanAs\\Documents\\Studie\\Thesis\\oude sets\\df-empty.csv")

In [7]:
df_model.columns.tolist()
df_model = df_model.drop(df_model.columns[0], axis=1)

In [8]:
from tqdm import tqdm


embeddings = []
for text in tqdm(df_model['descrip'], desc="Processing Texts"):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors="pt").to(device)
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    flattened_embedding = torch.mean(last_hidden_states, dim=1).flatten().detach().cpu().numpy().tolist()
    embeddings.append(flattened_embedding)

# Create a new column in the DataFrame for embeddings
df_model['embeddings'] = embeddings


Processing Texts:   0%|          | 104/72014 [01:48<20:50:26,  1.04s/it]


KeyboardInterrupt: 